In [1]:
# Author: Naveen Lalwani
# Script to distill knowledge from LeNet-300-100 trained on CIFAR-10 to student model

import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from keras.utils import np_utils
import time

Using TensorFlow backend.


In [ ]:
num_classes = 10
n_input = 3072
(x_train, y_train), (x_test, y_test) =  tf.keras.datasets.cifar10.load_data()

# Enabling One Hot Encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# Changing input image datatype to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizaig data
x_train  /= 255
x_test /= 255

x_train = x_train.reshape([50000, 3072])
x_test = x_test.reshape([10000, 3072])

In [ ]:
# Teacher Model: LeNet-300-100
def lenet_300_100_model():
    inputs = layers.Input(shape = (3072,))
    
    x = layers.Dense(300, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(100, activation='relu', name='FC2')(x)

    x = layers.Dense(10, name='logits')(x)
    preds = layers.Activation('softmax', name='Softmax')(x)

    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

#**Build Model LeNet-300-100**

In [4]:
model = lenet_300_100_model()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 300)               921900    
_________________________________________________________________
FC2 (Dense)                  (None, 100)               30100     
_________________________________________________________________
logits (Dense)               (None, 10)                1010      
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 953,010
Trainable params: 953,010
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size = 512) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
50000/50000 [==============================] - 13s 253us/sample - loss: 2.0006 - categorical_accuracy: 0.2807
Epoch 2/20
50000/50000 [==============================] - 12s 239us/sample - loss: 1.8121 - categorical_accuracy: 0.3603
Epoch 3/20
50000/50000 [==============================] - 12s 242us/sample - loss: 1.7368 - categorical_accuracy: 0.3877
Epoch 4/20
50000/50000 [==============================] - 12s 237us/sample - loss: 1.6725 - categorical_accuracy: 0.4106
Epoch 5/20
50000/50000 [==============================] - 13s 268us/sample - loss: 1.6206 - categorical_accuracy: 0.4288
Epoch 6/20
50000/50000 [==============================] - 12s 246us/sample - loss: 1.5734 - categorical_accuracy: 0.4465
Epoch 7/20
50000/50000 [==============================] - 12s 237us/sample - loss: 1.5423 - categorical_accuracy: 0.4528
Epoch 8/20
50000/50000 [==============================] - 12s 234us/sample - loss: 1.5134 - categorical_a

In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

10000/10000 [==============================] - 2s 200us/sample - loss: 1.4184 - categorical_accuracy: 0.5035
Test Loss: 1.4184400548934937
Test Accuracy: 0.5035


In [ ]:
getSoftmaxKnowledge = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = getSoftmaxKnowledge.predict(x_train)

In [ ]:
# Defining function described by Geoffrey Hinton in his paper of Knowledge Distillation
def softmax_with_temperature(logits, temperature):
    logits = logits / temperature
    return (np.exp(logits) / np.sum(np.exp(logits)))

In [ ]:
# Temperature is a hyperparameter
temperature = 2
softened_train_prob = softmax_with_temperature(model_logits, temperature)

In [ ]:
# Model Definition for the Student Model
def build_small_model():
    inputs = layers.Input(shape = (3072,))
    
    x = layers.Dense(50, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(10, name='logits')(x)
    
    preds = layers.Activation('softmax', name='Softmax')(x)
  
    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

In [48]:
small_model = build_small_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 50)                153650    
_________________________________________________________________
logits (Dense)               (None, 10)                510       
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 154,160
Trainable params: 154,160
Non-trainable params: 0
_________________________________________________________________


# **Distilling Knowledge in the student model**

In [49]:
# Optimization = Adam
# Loss = Cross Entropy loss
# Epochs = 50
# Trained with dark knowledge

small_model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['categorical_accuracy'])
small_model.fit(x_train, softened_train_prob, epochs=50, batch_size=128)

test_loss, test_acc = small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/50
50000/50000 [==============================] - 3s 60us/sample - loss: 3.9586e-05 - categorical_accuracy: 0.4213
Epoch 2/50
50000/50000 [==============================] - 3s 58us/sample - loss: 3.7143e-05 - categorical_accuracy: 0.5606
Epoch 3/50
50000/50000 [==============================] - 3s 58us/sample - loss: 3.6347e-05 - categorical_accuracy: 0.6235
Epoch 4/50
50000/50000 [==============================] - 3s 59us/sample - loss: 3.5920e-05 - categorical_accuracy: 0.6576
Epoch 5/50
50000/50000 [==============================] - 3s 59us/sample - loss: 3.5678e-05 - categorical_accuracy: 0.6822
Epoch 6/50
50000/50000 [==============================] - 3s 61us/sample - loss: 3.5531e-05 - categorical_accuracy: 0.6965
Epoch 7/50
50000/50000 [==============================] - 3s 59us/sample - loss: 3.5440e-05 - categorical_accuracy: 0.7084
Epoch 8/50
50000/50000 [==============================] - 3s 59us/sample - loss: 3.5359e-05 - categorical_accuracy: 0.7223
Epoch 9/50
50000

In [ ]:
small_model.save('model_50_LeNet-300-100_Distilled_CIFAR-10.h5')
model.save('model_LeNet-300-100_CIFAR10.h5')